In [3]:
import pygame
import numpy as np
import random
import time
import copy

pygame.init()

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
LIGHT_BLUE = (173, 216, 230)
DARK_BLUE = (70, 130, 180)
GREEN = (0, 200, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
PURPLE = (128, 0, 128)
ORIGINAL_CELL = (50, 50, 50)  
FONT = pygame.font.Font(None, 40)
SMALL_FONT = pygame.font.Font(None, 24)
SMALL_FONT2 = pygame.font.Font(None, 18)
TITLE_FONT = pygame.font.Font(None, 48)
SCREEN_WIDTH = 800 
SCREEN_HEIGHT = 700
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Sudoku Solver - Genetic & Backtracking Algorithms")
GRID_SIZE = 0
BOARD = None
ORIGINAL_BOARD = None  
BACKTRACKING_ITERATIONS = 0
SOLVING = False
ALGORITHM = None
GA_GENERATION = 0
GA_BEST_FITNESS = 0
GA_BEST_BOARD = None
GA_STEP_INTERVAL = 20  
ga_solver = None
SOLVING_SPEED = 2  
SOLUTION_FOUND = False
SOLUTION_TIME = 0
SOLUTION_ITERATIONS = 0
SOLUTION_GENERATIONS = 0
SOLUTION_ALGORITHM = None
START_TIME = 0

# --- DataSet ---
PREDEFINED_BOARDS = {
    "Easy 4x4": np.array([
        [1, 0, 0, 4],
        [0, 0, 1, 0],
        [0, 2, 0, 0],
        [4, 0, 0, 2]
    ]),
    "Medium 6x6": np.array([
        [0, 0, 1, 0, 0, 4],
        [0, 0, 0, 3, 0, 0],
        [0, 5, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0],
        [0, 4, 0, 0, 0, 0],
        [1, 0, 0, 5, 0, 0]
    ]),
    
    "Easy 9x9": np.array([
        [0, 4, 0, 1, 0, 0, 0, 5, 0],
        [1, 0, 7, 0, 0, 3, 9, 6, 0],
        [5, 2, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 7],
        [0, 0, 0, 9, 0, 6, 8, 0, 0],
        [8, 0, 3, 0, 5, 0, 6, 2, 0],
        [0, 9, 0, 0, 6, 0, 5, 4, 3],
        [6, 0, 0, 0, 8, 0, 7, 0, 0],
        [2, 5, 0, 0, 9, 7, 1, 0, 0]
    ]),
    "Medium 9x9": np.array([
        [0, 7, 0, 0, 0, 0, 0, 4, 3],
        [0, 4, 0, 0, 0, 9, 6, 1, 0],
        [8, 0, 0, 6, 3, 4, 9, 0, 0],
        [0, 9, 4, 0, 5, 2, 0, 0, 0],
        [3, 5, 8, 4, 6, 0, 0, 2, 0],
        [0, 0, 0, 8, 0, 0, 5, 3, 0],
        [0, 8, 0, 0, 7, 0, 0, 9, 1],
        [9, 0, 2, 1, 0, 0, 0, 0, 5],
        [0, 0, 7, 0, 4, 0, 8, 0, 2]
    ]),
    "Hard 9x9": np.array([
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ])
}

def is_valid(board, row, col, num, size):
    box_size = int(size**0.5)
    for i in range(size):
        if board[row][i] == num or board[i][col] == num:
            return False
    start_row, start_col = box_size * (row // box_size), box_size * (col // box_size)
    for i in range(start_row, start_row + box_size):
        for j in range(start_col, start_col + box_size):
            if board[i][j] == num:
                return False
    return True

def is_valid_sudoku(board, size):
    box_size = int(size**0.5)
    for row in range(size):
        row_values = [val for val in board[row] if val != 0]
        if len(row_values) != len(set(row_values)):
            return False
    for col in range(size):
        col_values = [board[row][col] for row in range(size) if board[row][col] != 0]
        if len(col_values) != len(set(col_values)):
            return False
    for box_row in range(0, size, box_size):
        for box_col in range(0, size, box_size):
            box_values = [
                board[row][col] 
                for row in range(box_row, box_row + box_size) 
                for col in range(box_col, box_col + box_size) 
                if board[row][col] != 0
            ]
            if len(box_values) != len(set(box_values)):
                return False
    
    return True

# -------------------- Backtracking Algorithm -------------------- #
def solve_backtracking_gui(board, size, screen):
    global BACKTRACKING_ITERATIONS, SOLVING, ALGORITHM, SOLVING_SPEED
    global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_ITERATIONS, SOLUTION_ALGORITHM, START_TIME
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            SOLVING = False
            return True  
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                SOLVING = False
                return True  
            elif event.key == pygame.K_UP and SOLVING_SPEED < 5:
                SOLVING_SPEED += 1
            elif event.key == pygame.K_DOWN and SOLVING_SPEED > 1:
                SOLVING_SPEED -= 1
    
    for row in range(size):
        for col in range(size):
            if board[row][col] == 0:
                for num in range(1, size + 1):
                    BACKTRACKING_ITERATIONS += 1
                    if is_valid(board, row, col, num, size):
                        board[row][col] = num
                        draw_board(screen, board, size, highlight=(row, col), algorithm=ALGORITHM, 
                                  iterations=BACKTRACKING_ITERATIONS)
                        pygame.display.flip()
                        pygame.time.delay(max(1, 500 // SOLVING_SPEED))  
                        
                        if solve_backtracking_gui(board, size, screen):
                            return True
                            
                        board[row][col] = 0
                        draw_board(screen, board, size, highlight=(row, col), algorithm=ALGORITHM, iterations=BACKTRACKING_ITERATIONS)
                        pygame.display.flip()
                        pygame.time.delay(max(1, 50 // SOLVING_SPEED))
                return False
    
    SOLUTION_FOUND = True
    SOLUTION_TIME = time.time() - START_TIME
    SOLUTION_ITERATIONS = BACKTRACKING_ITERATIONS
    SOLUTION_ALGORITHM = 'B'
    return True

# -------------------- Genetic Algorithm  -------------------- #
class SudokuGA_GUI:
    def __init__(self, puzzle, size, population_size=1000, mutation_rate=0.2, generations=3000, step_display_interval=50):
        self.puzzle = puzzle
        self.original_puzzle = puzzle.copy()  
        self.size = size
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.step_display_interval = step_display_interval
        self.sub_grid_size = int(np.sqrt(size))
        self.population = self.initialize_population()
        self.generation_count = 0
        self.best_fitness = 0
        self.best_solution = None
        self.stagnation_count = 0
        self.max_stagnation = 100  
        self.solution_found = False

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.generate_individual()
            population.append(individual)
        return population

    def generate_individual(self):
        individual = np.copy(self.puzzle)
        for row in range(self.size):
            missing_values = [i for i in range(1, self.size + 1) if i not in individual[row]]
            random.shuffle(missing_values)
            for col in range(self.size):
                if individual[row][col] == 0:
                    individual[row][col] = missing_values.pop()
        return individual

    def fitness(self, individual):
        row_score = sum(len(set(individual[row])) for row in range(self.size))
        col_score = sum(len(set(individual[:, col])) for col in range(self.size))
        sub_grid_score = sum(
            len(set(individual[row:row+self.sub_grid_size, col:col+self.sub_grid_size].flatten()))
            for row in range(0, self.size, self.sub_grid_size)
            for col in range(0, self.size, self.sub_grid_size)
        )
        return row_score + col_score + sub_grid_score

    def select_parents(self, population):
        scores = [(self.fitness(ind), ind) for ind in population]
        scores.sort(reverse=True, key=lambda x: x[0])
        return [ind for _, ind in scores[:self.population_size // 2]]

    def crossover(self, parent1, parent2):
        point = random.randint(0, self.size - 1)
        child = np.vstack((parent1[:point], parent2[point:]))
        return child

    def mutate(self, individual):
        for row in range(self.size):
            if random.random() < self.mutation_rate:
                idx = [i for i in range(self.size) if self.puzzle[row][i] == 0]
                if len(idx) > 1:
                    i1, i2 = random.sample(idx, 2)
                    individual[row][i1], individual[row][i2] = individual[row][i2], individual[row][i1]
        return individual

    def evolve(self, population):
        next_generation = []
        parents = self.select_parents(population)
        while len(next_generation) < self.population_size:
            p1, p2 = random.sample(parents, 2)
            child = self.crossover(p1, p2)
            child = self.mutate(child)
            next_generation.append(child)
        return next_generation

    
    def step(self):
        global GA_GENERATION, GA_BEST_FITNESS, GA_BEST_BOARD, SOLVING, SOLVING_SPEED
        global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_GENERATIONS, SOLUTION_ALGORITHM, START_TIME
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                SOLVING = False
                return "quit" 
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    SOLVING = False
                    return "cancel" 
                elif event.key == pygame.K_UP and SOLVING_SPEED < 5:
                    SOLVING_SPEED += 1
                elif event.key == pygame.K_DOWN and SOLVING_SPEED > 1:
                    SOLVING_SPEED -= 1

        max_possible_fitness = 3 * self.size * self.size
        if (self.generation_count >= self.generations or  self.best_fitness >= max_possible_fitness or self.stagnation_count >= self.max_stagnation):      
            SOLUTION_FOUND = True
            SOLUTION_TIME = time.time() - START_TIME
            SOLUTION_GENERATIONS = self.generation_count
            SOLUTION_ALGORITHM = 'G'
            
            SOLVING = False
            return "complete"  
            
        self.population = self.evolve(self.population)
        
        best_current = max(self.population, key=self.fitness)
        current_fitness = self.fitness(best_current)

        if current_fitness > self.best_fitness:
            self.best_fitness = current_fitness
            self.best_solution = best_current.copy()
            self.stagnation_count = 0
  
            if current_fitness >= max_possible_fitness:
                self.solution_found = True            
                SOLUTION_FOUND = True
                SOLUTION_TIME = time.time() - START_TIME
                SOLUTION_GENERATIONS = self.generation_count
                SOLUTION_ALGORITHM = 'G'
                
                SOLVING = False
                return "complete"
        else:
            self.stagnation_count += 1

        self.generation_count += 1
     
        if self.generation_count % max(1, self.step_display_interval // SOLVING_SPEED) == 0:
            GA_GENERATION = self.generation_count
            GA_BEST_FITNESS = self.best_fitness
            GA_BEST_BOARD = self.best_solution.copy() if self.best_solution is not None else None

        return False  

    def get_current_best(self):
        return self.best_solution if self.best_solution is not None else np.zeros((self.size, self.size), dtype=int)

# -------------------- GUI  -------------------- #
def draw_board(screen, board, size, highlight=None, algorithm=None, iterations=0, generation=0, fitness=0):
    screen.fill(WHITE)
    width = min(SCREEN_WIDTH, 600)  
    height = 500  
    cell_size = min(width // size, height // size)
    grid_width = cell_size * size
    grid_height = cell_size * size
    grid_x = (SCREEN_WIDTH - grid_width) // 2
    grid_y = 20  
    box_size = int(size**0.5)
    title = TITLE_FONT.render("Sudoku Solver", True, DARK_BLUE)
    screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 10))
    pygame.draw.rect(screen, LIGHT_BLUE, (grid_x - 5, grid_y - 5, grid_width + 10, grid_height + 10))
    
    for row in range(size):
        for col in range(size):
            cell_rect = pygame.Rect(grid_x + col * cell_size, grid_y + row * cell_size, cell_size, cell_size)
            box_row, box_col = row // box_size, col // box_size
            if (box_row + box_col) % 2 == 0:
                pygame.draw.rect(screen, WHITE, cell_rect)
            else:
                pygame.draw.rect(screen, (240, 240, 255), cell_rect)

            if ORIGINAL_BOARD is not None and ORIGINAL_BOARD[row][col] != 0:
                pygame.draw.rect(screen, (220, 255, 220), cell_rect)
            
            if highlight == (row, col):
                pygame.draw.rect(screen, YELLOW, cell_rect)

            value = board[row][col]
            if value != 0:
                if ORIGINAL_BOARD is not None and ORIGINAL_BOARD[row][col] != 0:
                    text_color = ORIGINAL_CELL
                else:
                    text_color = BLACK
                
                text = FONT.render(str(value), True, text_color)
                text_rect = text.get_rect(center=(grid_x + col * cell_size + cell_size // 2, grid_y + row * cell_size + cell_size // 2))
                screen.blit(text, text_rect)

    for i in range(size + 1):
        line_thickness = 3 if i % box_size == 0 else 1
        pygame.draw.line(screen, BLACK, 
                        (grid_x, grid_y + i * cell_size), 
                        (grid_x + grid_width, grid_y + i * cell_size), 
                        line_thickness)
        pygame.draw.line(screen, BLACK, 
                        (grid_x + i * cell_size, grid_y), 
                        (grid_x + i * cell_size, grid_y + grid_height), 
                        line_thickness)
    info_y = grid_y + grid_height + 20
    
    if algorithm == 'B' or SOLUTION_ALGORITHM == 'B':
        text = SMALL_FONT.render(f"Algorithm: Backtracking", True, BLACK)
        screen.blit(text, (20, info_y))
        iter_count = iterations if algorithm == 'B' else SOLUTION_ITERATIONS
        text = SMALL_FONT.render(f"Iterations: {iter_count}", True, BLACK)
        screen.blit(text, (20, info_y + 25))
        if algorithm == 'B':
            text = SMALL_FONT.render(f"Speed: {SOLVING_SPEED}/5 (arrows to adjust)", True, BLACK)
            screen.blit(text, (20, info_y + 50))

        if SOLUTION_FOUND and SOLUTION_ALGORITHM == 'B':
            text = SMALL_FONT.render(f"Solution found successfully at iteration {SOLUTION_ITERATIONS}!", True, GREEN)
            screen.blit(text, (20, info_y + 65))
            text = SMALL_FONT.render(f"Time taken: {SOLUTION_TIME:.2f} seconds", True, BLACK)
            screen.blit(text, (580, info_y))
            
    elif algorithm == 'G' or SOLUTION_ALGORITHM == 'G':
        text = SMALL_FONT.render(f"Algorithm: Genetic Algorithm", True, BLACK)
        screen.blit(text, (20, info_y))
        gen_count = generation if algorithm == 'G' else SOLUTION_GENERATIONS
        text = SMALL_FONT.render(f"Generation: {gen_count}", True, BLACK)
        screen.blit(text, (20, info_y + 25))
        fit_value = fitness if algorithm == 'G' else GA_BEST_FITNESS
        text = SMALL_FONT.render(f"Best Fitness: {fit_value}", True, BLACK)
        screen.blit(text, (20, info_y + 50))
        
        if algorithm == 'G':
            text = SMALL_FONT.render(f"Speed: {SOLVING_SPEED}/5 (arrows to adjust)", True, BLACK)
            screen.blit(text, (20, info_y + 75))

        if SOLUTION_FOUND and SOLUTION_ALGORITHM == 'G':
            text = SMALL_FONT.render(f"Solution found successfully at generation {SOLUTION_GENERATIONS}!", True, GREEN)
            screen.blit(text, (20, info_y + 75))
            text = SMALL_FONT.render(f"Time taken: {SOLUTION_TIME:.2f} seconds", True, BLACK)
            screen.blit(text, (580, info_y))

    button_width = 150
    button_height = 40
    button_y = SCREEN_HEIGHT - button_height - 20
    button_spacing = 20
    solve_backtrack_button = pygame.Rect(20, button_y, button_width, button_height)
    solve_genetic_button = pygame.Rect(20 + button_width + button_spacing, button_y, button_width, button_height)
    reset_button = pygame.Rect(SCREEN_WIDTH - button_width - 20, button_y, button_width, button_height)
    def draw_button(rect, color, text):
        pygame.draw.rect(screen, color, rect, border_radius=10)
        lighter_color = tuple(min(255, c + 30) for c in color)
        pygame.draw.rect(screen, lighter_color,pygame.Rect(rect.x, rect.y, rect.width, rect.height // 2), border_radius=10)
        text_surf = SMALL_FONT.render(text, True, WHITE)
        text_rect = text_surf.get_rect(center=rect.center)
        screen.blit(text_surf, text_rect)
        pygame.draw.rect(screen, BLACK, rect, 2, border_radius=10)      
        return rect

    backtrack_btn = draw_button(solve_backtrack_button, GREEN, "Solve (Backtrack)")
    genetic_btn = draw_button(solve_genetic_button, PURPLE, "Solve (Genetic)")
    reset_btn = draw_button(reset_button, RED, "Reset Board")
    help_text = SMALL_FONT2.render("Press ESC to cancel solving", True, BLACK)
    screen.blit(help_text, (SCREEN_WIDTH // 2 - help_text.get_width() // 2, button_y - 70))
    return backtrack_btn, genetic_btn, reset_btn

def handle_input_board(screen, size):
    board = np.zeros((size, size), dtype=int)
    inputting = True
    row, col = 0, 0
    cell_size = min(500, SCREEN_WIDTH - 100) // size
    grid_width = cell_size * size
    grid_x = (SCREEN_WIDTH - grid_width) // 2
    grid_y = 50
 
    instructions = [
        "Use number keys 1 to the chosen grid size to enter values",
        "Use 0 or Space to clear a cell",
        "Arrow keys to navigate",
        "Enter to finish, Escape to cancel"
    ]

    while inputting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return None
            if event.type == pygame.KEYDOWN:
                if pygame.K_1 <= event.key <= pygame.K_9 and 1 <= int(pygame.key.name(event.key)) <= size:
                    board[row][col] = int(pygame.key.name(event.key))
                    col = (col + 1) % size
                    if col == 0:
                        row = (row + 1) % size
                elif event.key == pygame.K_0 or event.key == pygame.K_SPACE:
                    board[row][col] = 0
                    col = (col + 1) % size
                    if col == 0:
                        row = (row + 1) % size
                elif event.key == pygame.K_LEFT:
                    col = (col - 1 + size) % size
                elif event.key == pygame.K_RIGHT:
                    col = (col + 1) % size
                elif event.key == pygame.K_UP:
                    row = (row - 1 + size) % size
                elif event.key == pygame.K_DOWN:
                    row = (row + 1) % size
                elif event.key == pygame.K_RETURN:
                    if is_valid_sudoku(board, size):
                        inputting = False
                    else:
                        pygame.draw.rect(screen, RED, (0, 0, SCREEN_WIDTH, SCREEN_HEIGHT))
                        pygame.display.flip()
                        pygame.time.delay(200)
                elif event.key == pygame.K_ESCAPE:
                    return None 
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                if grid_x <= mouse_x < grid_x + grid_width and grid_y <= mouse_y < grid_y + grid_width:
                    col = (mouse_x - grid_x) // cell_size
                    row = (mouse_y - grid_y) // cell_size
        screen.fill(WHITE)
        title = TITLE_FONT.render("Enter Sudoku Puzzle", True, DARK_BLUE)
        screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 10))
        pygame.draw.rect(screen, LIGHT_BLUE, (grid_x - 5, grid_y - 5, grid_width + 10, grid_width + 10)) 
        box_size = int(size**0.5)
        for r in range(size):
            for c in range(size):
                cell_rect = pygame.Rect(grid_x + c * cell_size, grid_y + r * cell_size, cell_size, cell_size)
                box_row, box_col = r // box_size, c // box_size
                if (box_row + box_col) % 2 == 0:
                    pygame.draw.rect(screen, WHITE, cell_rect)
                else:
                    pygame.draw.rect(screen, (240, 240, 255), cell_rect)
                
                if r == row and c == col:
                    pygame.draw.rect(screen, YELLOW, cell_rect)

                if board[r][c] != 0:
                    text = FONT.render(str(board[r][c]), True, BLACK)
                    text_rect = text.get_rect(center=(grid_x + c * cell_size + cell_size // 2, grid_y + r * cell_size + cell_size // 2))
                    screen.blit(text, text_rect)
        for i in range(size + 1):
            line_thickness = 3 if i % box_size == 0 else 1
            pygame.draw.line(screen, BLACK, 
                            (grid_x, grid_y + i * cell_size), 
                            (grid_x + grid_width, grid_y + i * cell_size), 
                            line_thickness)
            pygame.draw.line(screen, BLACK, 
                            (grid_x + i * cell_size, grid_y), 
                            (grid_x + i * cell_size, grid_y + grid_width), 
                            line_thickness)
        for i, instruction in enumerate(instructions):
            text = SMALL_FONT.render(instruction, True, BLACK)
            screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, grid_y + grid_width + 20 + i * 25))
        pygame.display.flip()
    return board
def select_predefined_board(screen):
    keys = list(PREDEFINED_BOARDS.keys())
    selected_index = 0
    running = True
    title_y = 50
    option_start_y = 120
    option_height = 40
    
    while running:
        screen.fill(WHITE)
        title_text = TITLE_FONT.render("Select a Predefined Board:", True, DARK_BLUE)
        screen.blit(title_text, (SCREEN_WIDTH // 2 - title_text.get_width() // 2, title_y))
        for i, key in enumerate(keys):
            option_rect = pygame.Rect(SCREEN_WIDTH // 4, option_start_y + i * option_height, SCREEN_WIDTH // 2, 30)
            if i == selected_index:
                pygame.draw.rect(screen, LIGHT_BLUE, option_rect, border_radius=5)
                pygame.draw.rect(screen, DARK_BLUE, option_rect, 2, border_radius=5)
                text_color = DARK_BLUE
            else:
                pygame.draw.rect(screen, (240, 240, 240), option_rect, border_radius=5)
                pygame.draw.rect(screen, GRAY, option_rect, 1, border_radius=5)
                text_color = BLACK
                
            text = SMALL_FONT.render(f"{key}", True, text_color)
            screen.blit(text, (option_rect.centerx - text.get_width() // 2, option_rect.centery - text.get_height() // 2))
            size_text = SMALL_FONT.render(f"{PREDEFINED_BOARDS[key].shape[0]}×{PREDEFINED_BOARDS[key].shape[0]}", True, GRAY)
            screen.blit(size_text, (option_rect.right + 10, option_rect.centery - size_text.get_height() // 2))
        instructions = SMALL_FONT.render("Use arrows to navigate, Enter to select, Esc to cancel", True, BLACK)
        screen.blit(instructions, (SCREEN_WIDTH // 2 - instructions.get_width() // 2, SCREEN_HEIGHT - 50))
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return None, 0
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    selected_index = (selected_index - 1 + len(keys)) % len(keys)
                elif event.key == pygame.K_DOWN:
                    selected_index = (selected_index + 1) % len(keys)
                elif event.key == pygame.K_RETURN:
                    return PREDEFINED_BOARDS[keys[selected_index]].copy(), PREDEFINED_BOARDS[keys[selected_index]].shape[0]
                elif event.key == pygame.K_ESCAPE:
                    return None, 0
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = event.pos
                for i, key in enumerate(keys):
                    option_rect = pygame.Rect(SCREEN_WIDTH // 4, option_start_y + i * option_height, SCREEN_WIDTH // 2, 30)
                    if option_rect.collidepoint(mouse_pos):
                        if selected_index == i:  
                            return PREDEFINED_BOARDS[keys[i]].copy(), PREDEFINED_BOARDS[keys[i]].shape[0]
                        else:
                            selected_index = i

# -------------------- Main -------------------- #
def main():
    global SOLVING, BOARD, GRID_SIZE, BACKTRACKING_ITERATIONS, ALGORITHM, ga_solver
    global GA_GENERATION, GA_BEST_FITNESS, GA_BEST_BOARD, ORIGINAL_BOARD, SOLVING_SPEED
    global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_ITERATIONS, SOLUTION_GENERATIONS, SOLUTION_ALGORITHM, START_TIME
    input_method = None
    while input_method not in ['1', '2', 'q']:
        screen.fill(WHITE)
        title_shadow = TITLE_FONT.render("Sudoku Solver", True, GRAY)
        title = TITLE_FONT.render("Sudoku Solver", True, DARK_BLUE)
        screen.blit(title_shadow, (SCREEN_WIDTH // 2 - title.get_width() // 2 + 2, 52))
        screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 50))        
        subtitle = SMALL_FONT.render("Genetic Algorithm & Backtracking Visualization", True, BLACK)
        screen.blit(subtitle, (SCREEN_WIDTH // 2 - subtitle.get_width() // 2, 100))
        option_y = 180
        option_height = 60
        option_width = 300
        option1_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y, option_width, option_height)
        pygame.draw.rect(screen, GREEN, option1_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option1_rect, 2, border_radius=10)
        text = FONT.render("1. Input your own", True, WHITE)
        screen.blit(text, (option1_rect.centerx - text.get_width() // 2, option1_rect.centery - text.get_height() // 2))
        option2_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y + option_height + 20, option_width, option_height)
        pygame.draw.rect(screen, PURPLE, option2_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option2_rect, 2, border_radius=10)
        text = FONT.render("2. Use predefined", True, WHITE)
        screen.blit(text, (option2_rect.centerx - text.get_width() // 2, option2_rect.centery - text.get_height() // 2))
        option3_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y + 2 * (option_height + 20), option_width, option_height)
        pygame.draw.rect(screen, RED, option3_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option3_rect, 2, border_radius=10)
        text = FONT.render("Q. Quit", True, WHITE)
        screen.blit(text, (option3_rect.centerx - text.get_width() // 2, option3_rect.centery - text.get_height() // 2))
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    input_method = '1'
                elif event.key == pygame.K_2:
                    input_method = '2'
                elif event.key == pygame.K_q:
                    return
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = event.pos
                if option1_rect.collidepoint(mouse_pos):
                    input_method = '1'
                elif option2_rect.collidepoint(mouse_pos):
                    input_method = '2'
                elif option3_rect.collidepoint(mouse_pos):
                    return

    if input_method == '1':
        size_inputting = True
        size = 0
        while size_inputting:
            screen.fill(WHITE)
            title = TITLE_FONT.render("Enter Grid Size", True, DARK_BLUE)
            screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 50))
            instructions = [
                "Enter a size",
                "Press Enter to continue, Escape to cancel"
            ]
            for i, instruction in enumerate(instructions):
                text = SMALL_FONT.render(instruction, True, BLACK)
                screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, 120 + i * 30))
            input_rect = pygame.Rect(SCREEN_WIDTH // 2 - 50, 200, 100, 40)
            pygame.draw.rect(screen, WHITE, input_rect)
            pygame.draw.rect(screen, DARK_BLUE, input_rect, 2)
            if size > 0:
                size_text = FONT.render(str(size), True, BLACK)
                screen.blit(size_text, (input_rect.centerx - size_text.get_width() // 2, 
                                      input_rect.centery - size_text.get_height() // 2))
            valid_sizes = SMALL_FONT.render("Valid sizes: 4, 6, 9", True, GRAY)
            screen.blit(valid_sizes, (SCREEN_WIDTH // 2 - valid_sizes.get_width() // 2, 260))
            
            pygame.display.flip()
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return
                if event.type == pygame.KEYDOWN:
                    if pygame.K_0 <= event.key <= pygame.K_9:
                        digit = int(pygame.key.name(event.key))
                        size = size * 10 + digit
                        if size > 25:
                            size = 25
                    elif event.key == pygame.K_RETURN and size > 0:
                        if size in [4, 6, 9]:
                            GRID_SIZE = size
                            BOARD = handle_input_board(screen, GRID_SIZE)
                            if BOARD is not None:
                                ORIGINAL_BOARD = BOARD.copy()  
                            size_inputting = False
                        else:
                            pygame.draw.rect(screen, RED, input_rect)
                            pygame.display.flip()
                            pygame.time.delay(200)

                    elif event.key == pygame.K_BACKSPACE:
                        size //= 10
                    elif event.key == pygame.K_ESCAPE:
                        return 
        
        if BOARD is None:
            return 

    elif input_method == '2':
        BOARD, GRID_SIZE = select_predefined_board(screen)
        if BOARD is None:
            return 
        ORIGINAL_BOARD = BOARD.copy() 

    if BOARD is None or GRID_SIZE == 0:
        return

    SOLVING = False
    ALGORITHM = None
    BACKTRACKING_ITERATIONS = 0
    GA_GENERATION = 0
    GA_BEST_FITNESS = 0
    GA_BEST_BOARD = None
    SOLVING_SPEED = 1
    SOLUTION_FOUND = False
    SOLUTION_TIME = 0
    SOLUTION_ITERATIONS = 0
    SOLUTION_GENERATIONS = 0
    SOLUTION_ALGORITHM = None
    solve_backtrack_button, solve_genetic_button, reset_button = draw_board(screen, BOARD, GRID_SIZE)
    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    if SOLVING:
                        SOLVING = False
                        ALGORITHM = None
                        BOARD = ORIGINAL_BOARD.copy()
                elif event.key == pygame.K_UP and SOLVING_SPEED < 5:
                    SOLVING_SPEED += 1
                elif event.key == pygame.K_DOWN and SOLVING_SPEED > 1:
                    SOLVING_SPEED -= 1
            if event.type == pygame.MOUSEBUTTONDOWN:
                if not SOLVING:
                    if solve_backtrack_button.collidepoint(event.pos):
                        SOLVING = True
                        ALGORITHM = 'B'
                        BACKTRACKING_ITERATIONS = 0
                        SOLUTION_FOUND = False
                        START_TIME = time.time()
                        board_copy = BOARD.copy()
                        if solve_backtracking_gui(board_copy, GRID_SIZE, screen):
                            BOARD = board_copy
                        SOLVING = False
                        ALGORITHM = None
                    elif solve_genetic_button.collidepoint(event.pos):
                        SOLVING = True
                        ALGORITHM = 'G'
                        GA_GENERATION = 0
                        GA_BEST_FITNESS = 0
                        GA_BEST_BOARD = None
                        SOLUTION_FOUND = False
                        START_TIME = time.time()
                        ga_solver = SudokuGA_GUI(BOARD.copy(), GRID_SIZE, step_display_interval=GA_STEP_INTERVAL)
                    elif reset_button.collidepoint(event.pos):
                        SOLVING = False
                        ALGORITHM = None
                        SOLUTION_FOUND = False
                        BACKTRACKING_ITERATIONS = 0
                        SOLUTION_TIME = 0
                        SOLUTION_ITERATIONS = 0
                        SOLUTION_GENERATIONS = 0
                        SOLUTION_ALGORITHM = None
                        GA_GENERATION = 0
                        GA_BEST_FITNESS = 0
                        GA_BEST_BOARD = None
                        SOLVING_SPEED = 1
                        if input_method == '1':
                            GRID_SIZE = 0
                            BOARD = None
                            main() 
                            return
                        elif input_method == '2':
                            BOARD, GRID_SIZE = select_predefined_board(screen)
                            if BOARD is None:
                                return
                            ORIGINAL_BOARD = BOARD.copy()
                else:
                    if reset_button.collidepoint(event.pos):
                        SOLVING = False
                        ALGORITHM = None
                        if input_method == '1':
                            GRID_SIZE = 0
                            BOARD = None
                            main()
                            return
                        elif input_method == '2':
                            BOARD, GRID_SIZE = select_predefined_board(screen)
                            if BOARD is None:
                                return
                            ORIGINAL_BOARD = BOARD.copy()

        if SOLVING and ALGORITHM == 'G':
            result = ga_solver.step()
            if result == False:
                GA_GENERATION = ga_solver.generation_count
                GA_BEST_FITNESS = ga_solver.best_fitness
                GA_BEST_BOARD = ga_solver.get_current_best()
                draw_board(screen, GA_BEST_BOARD, GRID_SIZE, algorithm=ALGORITHM, 
                      generation=GA_GENERATION, fitness=GA_BEST_FITNESS)
                pygame.display.flip()
                pygame.time.delay(max(1, 500 // SOLVING_SPEED))
            else:
                SOLVING = False
                ALGORITHM = None    
                if result == "complete":
                    BOARD = ga_solver.get_current_best()
                elif result == "cancel":
                    BOARD = ORIGINAL_BOARD.copy()   
                draw_board(screen, BOARD, GRID_SIZE)
                pygame.display.flip()
        elif not SOLVING:
            draw_board(screen, BOARD, GRID_SIZE)
            pygame.display.flip()

    pygame.quit()

if __name__ == '__main__':
    main()

In [9]:
# FINALE

import pygame
import numpy as np
import random
import time
import copy

# Initialize Pygame
pygame.init()

# --- Colors ---
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
LIGHT_BLUE = (173, 216, 230)
DARK_BLUE = (70, 130, 180)
GREEN = (0, 200, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
PURPLE = (128, 0, 128)
ORIGINAL_CELL = (50, 50, 50)  # Color for original cells

# --- Fonts ---
FONT = pygame.font.Font(None, 40)
SMALL_FONT = pygame.font.Font(None, 24)
SMALL_FONT2 = pygame.font.Font(None, 18)

TITLE_FONT = pygame.font.Font(None, 48)

# --- Screen Dimensions ---
SCREEN_WIDTH = 800  # Increased width for better UI
SCREEN_HEIGHT = 700
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Sudoku Solver - Genetic & Backtracking Algorithms")

# --- Board and Game Logic Variables ---
GRID_SIZE = 0
BOARD = None
ORIGINAL_BOARD = None  # To track original cells
BACKTRACKING_ITERATIONS = 0
SOLVING = False
ALGORITHM = None
GA_GENERATION = 0
GA_BEST_FITNESS = 0
GA_BEST_BOARD = None
GA_STEP_INTERVAL = 20  # Faster visualization
ga_solver = None
SOLVING_SPEED = 2  # Speed multiplier (1-5)

# --- Solution Information ---
SOLUTION_FOUND = False
SOLUTION_TIME = 0
SOLUTION_ITERATIONS = 0
SOLUTION_GENERATIONS = 0
SOLUTION_ALGORITHM = None
START_TIME = 0

# --- Predefined Boards (moved here for GUI access) ---
PREDEFINED_BOARDS = {
    "Easy 4x4": np.array([
        [1, 0, 0, 4],
        [0, 0, 1, 0],
        [0, 2, 0, 0],
        [4, 0, 0, 2]
    ]),
    "Medium 6x6": np.array([
        [0, 0, 1, 0, 0, 4],
        [0, 0, 0, 3, 0, 0],
        [0, 5, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0],
        [0, 4, 0, 0, 0, 0],
        [1, 0, 0, 5, 0, 0]
    ]),
    
    "Easy 9x9": np.array([
        [0, 4, 0, 1, 0, 0, 0, 5, 0],
        [1, 0, 7, 0, 0, 3, 9, 6, 0],
        [5, 2, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 7],
        [0, 0, 0, 9, 0, 6, 8, 0, 0],
        [8, 0, 3, 0, 5, 0, 6, 2, 0],
        [0, 9, 0, 0, 6, 0, 5, 4, 3],
        [6, 0, 0, 0, 8, 0, 7, 0, 0],
        [2, 5, 0, 0, 9, 7, 1, 0, 0]
    ]),
    "Medium 9x9": np.array([
        [0, 7, 0, 0, 0, 0, 0, 4, 3],
        [0, 4, 0, 0, 0, 9, 6, 1, 0],
        [8, 0, 0, 6, 3, 4, 9, 0, 0],
        [0, 9, 4, 0, 5, 2, 0, 0, 0],
        [3, 5, 8, 4, 6, 0, 0, 2, 0],
        [0, 0, 0, 8, 0, 0, 5, 3, 0],
        [0, 8, 0, 0, 7, 0, 0, 9, 1],
        [9, 0, 2, 1, 0, 0, 0, 0, 5],
        [0, 0, 7, 0, 4, 0, 8, 0, 2]
    ]),
    "Hard 9x9": np.array([
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ])
}

# -------------------- Helper Functions -------------------- #
def is_valid(board, row, col, num, size):
    box_size = int(size**0.5)
    # Check row and column
    for i in range(size):
        if board[row][i] == num or board[i][col] == num:
            return False
    # Check box
    start_row, start_col = box_size * (row // box_size), box_size * (col // box_size)
    for i in range(start_row, start_row + box_size):
        for j in range(start_col, start_col + box_size):
            if board[i][j] == num:
                return False
    return True

def is_valid_sudoku(board, size):
    """Check if the current board is valid (no duplicates in rows, columns, boxes)"""
    box_size = int(size**0.5)
    
    # Check rows
    for row in range(size):
        row_values = [val for val in board[row] if val != 0]
        if len(row_values) != len(set(row_values)):
            return False
    
    # Check columns
    for col in range(size):
        col_values = [board[row][col] for row in range(size) if board[row][col] != 0]
        if len(col_values) != len(set(col_values)):
            return False
    
    # Check boxes
    for box_row in range(0, size, box_size):
        for box_col in range(0, size, box_size):
            box_values = [
                board[row][col] 
                for row in range(box_row, box_row + box_size) 
                for col in range(box_col, box_col + box_size) 
                if board[row][col] != 0
            ]
            if len(box_values) != len(set(box_values)):
                return False
    
    return True

# -------------------- Backtracking Solver (GUI Integration) -------------------- #
def solve_backtracking_gui(board, size, screen):
    global BACKTRACKING_ITERATIONS, SOLVING, ALGORITHM, SOLVING_SPEED
    global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_ITERATIONS, SOLUTION_ALGORITHM, START_TIME
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            SOLVING = False
            return True  # Indicate quit
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                SOLVING = False
                return True  # User canceled
            elif event.key == pygame.K_UP and SOLVING_SPEED < 5:
                SOLVING_SPEED += 1
            elif event.key == pygame.K_DOWN and SOLVING_SPEED > 1:
                SOLVING_SPEED -= 1
    
    for row in range(size):
        for col in range(size):
            if board[row][col] == 0:
                for num in range(1, size + 1):
                    BACKTRACKING_ITERATIONS += 1
                    if is_valid(board, row, col, num, size):
                        board[row][col] = num
                        draw_board(screen, board, size, highlight=(row, col), algorithm=ALGORITHM, 
                                  iterations=BACKTRACKING_ITERATIONS)
                        pygame.display.flip()
                        pygame.time.delay(max(1, 500 // SOLVING_SPEED))  # Adjustable delay
                        
                        if solve_backtracking_gui(board, size, screen):
                            return True
                            
                        board[row][col] = 0
                        draw_board(screen, board, size, highlight=(row, col), algorithm=ALGORITHM, 
                                  iterations=BACKTRACKING_ITERATIONS)
                        pygame.display.flip()
                        pygame.time.delay(max(1, 50 // SOLVING_SPEED))
                return False
    
    # Solution found
    SOLUTION_FOUND = True
    SOLUTION_TIME = time.time() - START_TIME
    SOLUTION_ITERATIONS = BACKTRACKING_ITERATIONS
    SOLUTION_ALGORITHM = 'B'
    return True

# -------------------- Genetic Algorithm Solver (GUI Integration) -------------------- #
class SudokuGA_GUI:
    def __init__(self, puzzle, size, population_size=1000, mutation_rate=0.2, generations=3000, step_display_interval=50):
        self.puzzle = puzzle
        self.original_puzzle = puzzle.copy()  # Keep a copy of the original puzzle
        self.size = size
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.step_display_interval = step_display_interval
        self.sub_grid_size = int(np.sqrt(size))
        self.population = self.initialize_population()
        self.generation_count = 0
        self.best_fitness = 0
        self.best_solution = None
        self.stagnation_count = 0
        self.max_stagnation = 100  # Stop if no improvement for this many generations
        self.solution_found = False

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.generate_individual()
            population.append(individual)
        return population

    def generate_individual(self):
        individual = np.copy(self.puzzle)
        for row in range(self.size):
            missing_values = [i for i in range(1, self.size + 1) if i not in individual[row]]
            random.shuffle(missing_values)
            for col in range(self.size):
                if individual[row][col] == 0:
                    individual[row][col] = missing_values.pop()
        return individual

    def fitness(self, individual):
        row_score = sum(len(set(individual[row])) for row in range(self.size))
        col_score = sum(len(set(individual[:, col])) for col in range(self.size))
        sub_grid_score = sum(
            len(set(individual[row:row+self.sub_grid_size, col:col+self.sub_grid_size].flatten()))
            for row in range(0, self.size, self.sub_grid_size)
            for col in range(0, self.size, self.sub_grid_size)
        )
        return row_score + col_score + sub_grid_score

    def select_parents(self, population):
        scores = [(self.fitness(ind), ind) for ind in population]
        scores.sort(reverse=True, key=lambda x: x[0])
        return [ind for _, ind in scores[:self.population_size // 2]]

    def crossover(self, parent1, parent2):
        point = random.randint(0, self.size - 1)
        child = np.vstack((parent1[:point], parent2[point:]))
        return child

    def mutate(self, individual):
        for row in range(self.size):
            if random.random() < self.mutation_rate:
                idx = [i for i in range(self.size) if self.puzzle[row][i] == 0]
                if len(idx) > 1:
                    i1, i2 = random.sample(idx, 2)
                    individual[row][i1], individual[row][i2] = individual[row][i2], individual[row][i1]
        return individual

    def evolve(self, population):
        next_generation = []
        parents = self.select_parents(population)
        while len(next_generation) < self.population_size:
            p1, p2 = random.sample(parents, 2)
            child = self.crossover(p1, p2)
            child = self.mutate(child)
            next_generation.append(child)
        return next_generation

    
    def step(self):
        global GA_GENERATION, GA_BEST_FITNESS, GA_BEST_BOARD, SOLVING, SOLVING_SPEED
        global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_GENERATIONS, SOLUTION_ALGORITHM, START_TIME
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                SOLVING = False
                return "quit"  # Indicate quit
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    SOLVING = False
                    return "Cancel"  # User canceled
                elif event.key == pygame.K_UP and SOLVING_SPEED < 5:
                    SOLVING_SPEED += 1
                elif event.key == pygame.K_DOWN and SOLVING_SPEED > 1:
                    SOLVING_SPEED -= 1

        # Check if we've reached the maximum generations or found a perfect solution
        max_possible_fitness = 3 * self.size * self.size
        if (self.generation_count >= self.generations or 
            self.best_fitness >= max_possible_fitness or 
            self.stagnation_count >= self.max_stagnation):
            
            # Record solution information
            SOLUTION_FOUND = True
            SOLUTION_TIME = time.time() - START_TIME
            SOLUTION_GENERATIONS = self.generation_count
            SOLUTION_ALGORITHM = 'G'
            
            SOLVING = False
            return "Complete"  # Indicate completion
            
        # Evolve the population
        self.population = self.evolve(self.population)
        
        # Find the best individual in this generation
        best_current = max(self.population, key=self.fitness)
        current_fitness = self.fitness(best_current)

        # Update best solution if improved
        if current_fitness > self.best_fitness:
            self.best_fitness = current_fitness
            self.best_solution = best_current.copy()
            self.stagnation_count = 0
            
            # Check if we've found a perfect solution
            if current_fitness >= max_possible_fitness:
                self.solution_found = True
                
                # Record solution information
                SOLUTION_FOUND = True
                SOLUTION_TIME = time.time() - START_TIME
                SOLUTION_GENERATIONS = self.generation_count
                SOLUTION_ALGORITHM = 'G'
                
                SOLVING = False
                return "Complete"
        else:
            self.stagnation_count += 1

        self.generation_count += 1
        
        # Only update display occasionally for performance
        if self.generation_count % max(1, self.step_display_interval // SOLVING_SPEED) == 0:
            GA_GENERATION = self.generation_count
            GA_BEST_FITNESS = self.best_fitness
            GA_BEST_BOARD = self.best_solution.copy() if self.best_solution is not None else None
            
        return False  # Continue evolving

    def get_current_best(self):
        return self.best_solution if self.best_solution is not None else np.zeros((self.size, self.size), dtype=int)

# -------------------- GUI Drawing Functions -------------------- #
def draw_board(screen, board, size, highlight=None, algorithm=None, iterations=0, generation=0, fitness=0):
    screen.fill(WHITE)
    width = min(SCREEN_WIDTH, 600)  # Max width for the grid
    height = 500  # Height for the grid
    cell_size = min(width // size, height // size)
    grid_width = cell_size * size
    grid_height = cell_size * size
    
    # Center the grid
    grid_x = (SCREEN_WIDTH - grid_width) // 2
    grid_y = 20
    
    box_size = int(size**0.5)

    # Draw title
    title = TITLE_FONT.render("Sudoku Solver", True, DARK_BLUE)
    screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 10))

    # Draw grid background
    pygame.draw.rect(screen, LIGHT_BLUE, (grid_x - 5, grid_y - 5, grid_width + 10, grid_height + 10))
    
    # Draw cells and values
    for row in range(size):
        for col in range(size):
            # Cell background
            cell_rect = pygame.Rect(grid_x + col * cell_size, grid_y + row * cell_size, cell_size, cell_size)
            
            # Alternating colors for boxes
            box_row, box_col = row // box_size, col // box_size
            if (box_row + box_col) % 2 == 0:
                pygame.draw.rect(screen, WHITE, cell_rect)
            else:
                pygame.draw.rect(screen, (240, 240, 255), cell_rect)
                
            # Highlight original cells
            if ORIGINAL_BOARD is not None and ORIGINAL_BOARD[row][col] != 0:
                pygame.draw.rect(screen, (220, 255, 220), cell_rect)
            
            # Highlight current cell
            if highlight == (row, col):
                pygame.draw.rect(screen, YELLOW, cell_rect)
            
            # Draw cell value
            value = board[row][col]
            if value != 0:
                # Different color for original values
                if ORIGINAL_BOARD is not None and ORIGINAL_BOARD[row][col] != 0:
                    text_color = ORIGINAL_CELL
                else:
                    text_color = BLACK
                
                text = FONT.render(str(value), True, text_color)
                text_rect = text.get_rect(center=(grid_x + col * cell_size + cell_size // 2, 
                                                grid_y + row * cell_size + cell_size // 2))
                screen.blit(text, text_rect)
    
    # Draw grid lines
    for i in range(size + 1):
        line_thickness = 3 if i % box_size == 0 else 1
        # Horizontal lines
        pygame.draw.line(screen, BLACK, 
                        (grid_x, grid_y + i * cell_size), 
                        (grid_x + grid_width, grid_y + i * cell_size), 
                        line_thickness)
        # Vertical lines
        pygame.draw.line(screen, BLACK, 
                        (grid_x + i * cell_size, grid_y), 
                        (grid_x + i * cell_size, grid_y + grid_height), 
                        line_thickness)

    # Display algorithm information
    info_y = grid_y + grid_height + 20
    
    # Always show algorithm information if we have it
    if algorithm == 'B' or SOLUTION_ALGORITHM == 'B':
        text = SMALL_FONT.render(f"Algorithm: Backtracking", True, BLACK)
        screen.blit(text, (20, info_y))
        
        # Show iterations
        iter_count = iterations if algorithm == 'B' else SOLUTION_ITERATIONS
        text = SMALL_FONT.render(f"Iterations: {iter_count}", True, BLACK)
        screen.blit(text, (20, info_y + 25))
        
        # Show solving speed if still solving
        if algorithm == 'B':
            text = SMALL_FONT.render(f"Speed: {SOLVING_SPEED}/5 (arrows to adjust)", True, BLACK)
            screen.blit(text, (20, info_y + 50))
        
        # Show solution message if found
        if SOLUTION_FOUND and SOLUTION_ALGORITHM == 'B':
            text = SMALL_FONT.render(f"Solution found successfully at iteration {SOLUTION_ITERATIONS}!", True, GREEN)
            screen.blit(text, (20, info_y + 65))
            text = SMALL_FONT.render(f"Time taken: {SOLUTION_TIME:.2f} seconds", True, BLACK)
            screen.blit(text, (580, info_y))
            
    elif algorithm == 'G' or SOLUTION_ALGORITHM == 'G':
        text = SMALL_FONT.render(f"Algorithm: Genetic Algorithm", True, BLACK)
        screen.blit(text, (20, info_y))
        
        # Show generation count
        gen_count = generation if algorithm == 'G' else SOLUTION_GENERATIONS
        text = SMALL_FONT.render(f"Generation: {gen_count}", True, BLACK)
        screen.blit(text, (20, info_y + 25))
        
        # Show fitness
        fit_value = fitness if algorithm == 'G' else GA_BEST_FITNESS
        text = SMALL_FONT.render(f"Best Fitness: {fit_value}", True, BLACK)
        screen.blit(text, (20, info_y + 50))
        
        
        # Show solution message if found
        if SOLUTION_FOUND and SOLUTION_ALGORITHM == 'G':
            text = SMALL_FONT.render(f"Solution found successfully at generation {SOLUTION_GENERATIONS}!", True, GREEN)
            screen.blit(text, (20, info_y + 75))
            text = SMALL_FONT.render(f"Time taken: {SOLUTION_TIME:.2f} seconds", True, BLACK)
            screen.blit(text, (580, info_y))
            
        
    # Draw buttons
    button_width = 150
    button_height = 40
    button_y = SCREEN_HEIGHT - button_height - 20
    button_spacing = 20

    # Calculate button positions
    solve_backtrack_button = pygame.Rect(20, button_y, button_width, button_height)
    solve_genetic_button = pygame.Rect(20 + button_width + button_spacing, button_y, button_width, button_height)
    reset_button = pygame.Rect(SCREEN_WIDTH - button_width - 20, button_y, button_width, button_height)

    # Draw buttons with gradients and rounded corners
    def draw_button(rect, color, text):
        # Draw button with gradient
        pygame.draw.rect(screen, color, rect, border_radius=10)
        lighter_color = tuple(min(255, c + 30) for c in color)
        pygame.draw.rect(screen, lighter_color, 
                        pygame.Rect(rect.x, rect.y, rect.width, rect.height // 2), 
                        border_radius=10)
        
        # Draw text
        text_surf = SMALL_FONT.render(text, True, WHITE)
        text_rect = text_surf.get_rect(center=rect.center)
        screen.blit(text_surf, text_rect)
        
        # Draw border
        pygame.draw.rect(screen, BLACK, rect, 2, border_radius=10)
        
        return rect

    backtrack_btn = draw_button(solve_backtrack_button, GREEN, "Solve (Backtrack)")
    genetic_btn = draw_button(solve_genetic_button, PURPLE, "Solve (Genetic)")
    reset_btn = draw_button(reset_button, RED, "Reset Board")

    # Draw help text
    help_text = SMALL_FONT2.render("Press ESC to cancel solving", True, BLACK)
    screen.blit(help_text, (SCREEN_WIDTH // 2 - help_text.get_width() // 2, button_y - 70))

    return backtrack_btn, genetic_btn, reset_btn

def handle_input_board(screen, size):
    board = np.zeros((size, size), dtype=int)
    inputting = True
    row, col = 0, 0
    cell_size = min(500, SCREEN_WIDTH - 100) // size
    grid_width = cell_size * size
    grid_x = (SCREEN_WIDTH - grid_width) // 2
    grid_y = 50
    
    # Instructions
    instructions = [
        "Use number keys 1-9 to enter values",
        "Use 0 or Space to clear a cell",
        "Arrow keys to navigate",
        "Enter to finish, Escape to cancel"
    ]

    while inputting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return None
            if event.type == pygame.KEYDOWN:
                if pygame.K_1 <= event.key <= pygame.K_9 and 1 <= int(pygame.key.name(event.key)) <= size:
                    board[row][col] = int(pygame.key.name(event.key))
                    col = (col + 1) % size
                    if col == 0:
                        row = (row + 1) % size
                elif event.key == pygame.K_0 or event.key == pygame.K_SPACE:
                    board[row][col] = 0
                    col = (col + 1) % size
                    if col == 0:
                        row = (row + 1) % size
                elif event.key == pygame.K_LEFT:
                    col = (col - 1 + size) % size
                elif event.key == pygame.K_RIGHT:
                    col = (col + 1) % size
                elif event.key == pygame.K_UP:
                    row = (row - 1 + size) % size
                elif event.key == pygame.K_DOWN:
                    row = (row + 1) % size
                elif event.key == pygame.K_RETURN:
                    # Validate the board before accepting
                    if is_valid_sudoku(board, size):
                        inputting = False
                    else:
                        # Flash red to indicate invalid board
                        pygame.draw.rect(screen, RED, (0, 0, SCREEN_WIDTH, SCREEN_HEIGHT))
                        pygame.display.flip()
                        pygame.time.delay(200)
                elif event.key == pygame.K_ESCAPE:
                    return None # Cancel input
            
            # Handle mouse input
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                if grid_x <= mouse_x < grid_x + grid_width and grid_y <= mouse_y < grid_y + grid_width:
                    col = (mouse_x - grid_x) // cell_size
                    row = (mouse_y - grid_y) // cell_size

        screen.fill(WHITE)
        
        # Draw title
        title = TITLE_FONT.render("Enter Sudoku Puzzle", True, DARK_BLUE)
        screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 10))
        
        # Draw grid background
        pygame.draw.rect(screen, LIGHT_BLUE, (grid_x - 5, grid_y - 5, grid_width + 10, grid_width + 10))
        
        box_size = int(size**0.5)
        
        # Draw cells
        for r in range(size):
            for c in range(size):
                # Cell background
                cell_rect = pygame.Rect(grid_x + c * cell_size, grid_y + r * cell_size, cell_size, cell_size)
                
                # Alternating colors for boxes
                box_row, box_col = r // box_size, c // box_size
                if (box_row + box_col) % 2 == 0:
                    pygame.draw.rect(screen, WHITE, cell_rect)
                else:
                    pygame.draw.rect(screen, (240, 240, 255), cell_rect)
                
                # Highlight current cell
                if r == row and c == col:
                    pygame.draw.rect(screen, YELLOW, cell_rect)
                
                # Draw cell value
                if board[r][c] != 0:
                    text = FONT.render(str(board[r][c]), True, BLACK)
                    text_rect = text.get_rect(center=(grid_x + c * cell_size + cell_size // 2, 
                                                    grid_y + r * cell_size + cell_size // 2))
                    screen.blit(text, text_rect)
        
        # Draw grid lines
        for i in range(size + 1):
            line_thickness = 3 if i % box_size == 0 else 1
            # Horizontal lines
            pygame.draw.line(screen, BLACK, 
                            (grid_x, grid_y + i * cell_size), 
                            (grid_x + grid_width, grid_y + i * cell_size), 
                            line_thickness)
            # Vertical lines
            pygame.draw.line(screen, BLACK, 
                            (grid_x + i * cell_size, grid_y), 
                            (grid_x + i * cell_size, grid_y + grid_width), 
                            line_thickness)

        # Draw instructions
        for i, instruction in enumerate(instructions):
            text = SMALL_FONT.render(instruction, True, BLACK)
            screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, grid_y + grid_width + 20 + i * 25))

        pygame.display.flip()

    return board

def select_predefined_board(screen):
    keys = list(PREDEFINED_BOARDS.keys())
    selected_index = 0
    running = True
    
    # Calculate layout
    title_y = 50
    option_start_y = 120
    option_height = 40
    
    while running:
        screen.fill(WHITE)
        
        # Draw title
        title_text = TITLE_FONT.render("Select a Predefined Board:", True, DARK_BLUE)
        screen.blit(title_text, (SCREEN_WIDTH // 2 - title_text.get_width() // 2, title_y))

        # Draw options with better styling
        for i, key in enumerate(keys):
            option_rect = pygame.Rect(SCREEN_WIDTH // 4, option_start_y + i * option_height, SCREEN_WIDTH // 2, 30)
            
            # Highlight selected option
            if i == selected_index:
                pygame.draw.rect(screen, LIGHT_BLUE, option_rect, border_radius=5)
                pygame.draw.rect(screen, DARK_BLUE, option_rect, 2, border_radius=5)
                text_color = DARK_BLUE
            else:
                pygame.draw.rect(screen, (240, 240, 240), option_rect, border_radius=5)
                pygame.draw.rect(screen, GRAY, option_rect, 1, border_radius=5)
                text_color = BLACK
                
            text = SMALL_FONT.render(f"{key}", True, text_color)
            screen.blit(text, (option_rect.centerx - text.get_width() // 2, option_rect.centery - text.get_height() // 2))
            
            # Preview board size
            size_text = SMALL_FONT.render(f"{PREDEFINED_BOARDS[key].shape[0]}×{PREDEFINED_BOARDS[key].shape[0]}", True, GRAY)
            screen.blit(size_text, (option_rect.right + 10, option_rect.centery - size_text.get_height() // 2))

        # Draw instructions
        instructions = SMALL_FONT.render("Use arrows to navigate, Enter to select, Esc to cancel", True, BLACK)
        screen.blit(instructions, (SCREEN_WIDTH // 2 - instructions.get_width() // 2, SCREEN_HEIGHT - 50))
        
        pygame.display.flip()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return None, 0
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    selected_index = (selected_index - 1 + len(keys)) % len(keys)
                elif event.key == pygame.K_DOWN:
                    selected_index = (selected_index + 1) % len(keys)
                elif event.key == pygame.K_RETURN:
                    return PREDEFINED_BOARDS[keys[selected_index]].copy(), PREDEFINED_BOARDS[keys[selected_index]].shape[0]
                elif event.key == pygame.K_ESCAPE:
                    return None, 0
            
            # Handle mouse input
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = event.pos
                for i, key in enumerate(keys):
                    option_rect = pygame.Rect(SCREEN_WIDTH // 4, option_start_y + i * option_height, SCREEN_WIDTH // 2, 30)
                    if option_rect.collidepoint(mouse_pos):
                        if selected_index == i:  # Double click/select
                            return PREDEFINED_BOARDS[keys[i]].copy(), PREDEFINED_BOARDS[keys[i]].shape[0]
                        else:
                            selected_index = i

# -------------------- Main Game Loop -------------------- #
def main():
    global SOLVING, BOARD, GRID_SIZE, BACKTRACKING_ITERATIONS, ALGORITHM, ga_solver
    global GA_GENERATION, GA_BEST_FITNESS, GA_BEST_BOARD, ORIGINAL_BOARD, SOLVING_SPEED
    global SOLUTION_FOUND, SOLUTION_TIME, SOLUTION_ITERATIONS, SOLUTION_GENERATIONS, SOLUTION_ALGORITHM, START_TIME

    # Main menu
    input_method = None
    while input_method not in ['1', '2', 'q']:
        screen.fill(WHITE)
        
        # Draw title with shadow effect
        title_shadow = TITLE_FONT.render("Sudoku Solver", True, GRAY)
        title = TITLE_FONT.render("Sudoku Solver", True, DARK_BLUE)
        screen.blit(title_shadow, (SCREEN_WIDTH // 2 - title.get_width() // 2 + 2, 52))
        screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 50))
        
        subtitle = SMALL_FONT.render("Genetic Algorithm & Backtracking Visualization", True, BLACK)
        screen.blit(subtitle, (SCREEN_WIDTH // 2 - subtitle.get_width() // 2, 100))
        
        # Draw menu options
        option_y = 180
        option_height = 60
        option_width = 300
        
        # Option 1: Input your own
        option1_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y, option_width, option_height)
        pygame.draw.rect(screen, GREEN, option1_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option1_rect, 2, border_radius=10)
        text = FONT.render("1. Input your own", True, WHITE)
        screen.blit(text, (option1_rect.centerx - text.get_width() // 2, option1_rect.centery - text.get_height() // 2))
        
        # Option 2: Use predefined
        option2_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y + option_height + 20, option_width, option_height)
        pygame.draw.rect(screen, PURPLE, option2_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option2_rect, 2, border_radius=10)
        text = FONT.render("2. Use predefined", True, WHITE)
        screen.blit(text, (option2_rect.centerx - text.get_width() // 2, option2_rect.centery - text.get_height() // 2))
        
        # Option 3: Quit
        option3_rect = pygame.Rect(SCREEN_WIDTH // 2 - option_width // 2, option_y + 2 * (option_height + 20), option_width, option_height)
        pygame.draw.rect(screen, RED, option3_rect, border_radius=10)
        pygame.draw.rect(screen, BLACK, option3_rect, 2, border_radius=10)
        text = FONT.render("Q. Quit", True, WHITE)
        screen.blit(text, (option3_rect.centerx - text.get_width() // 2, option3_rect.centery - text.get_height() // 2))
        
       
        
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    input_method = '1'
                elif event.key == pygame.K_2:
                    input_method = '2'
                elif event.key == pygame.K_q:
                    return
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = event.pos
                if option1_rect.collidepoint(mouse_pos):
                    input_method = '1'
                elif option2_rect.collidepoint(mouse_pos):
                    input_method = '2'
                elif option3_rect.collidepoint(mouse_pos):
                    return

    if input_method == '1':
        size_inputting = True
        size = 0
        while size_inputting:
            screen.fill(WHITE)
            
            # Draw title
            title = TITLE_FONT.render("Enter Grid Size", True, DARK_BLUE)
            screen.blit(title, (SCREEN_WIDTH // 2 - title.get_width() // 2, 50))
            
            # Draw instructions
            instructions = [
                "Enter a size",
                "Press Enter to continue, Escape to cancel"
            ]
            for i, instruction in enumerate(instructions):
                text = SMALL_FONT.render(instruction, True, BLACK)
                screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, 120 + i * 30))
            
            # Draw input box
            input_rect = pygame.Rect(SCREEN_WIDTH // 2 - 50, 200, 100, 40)
            pygame.draw.rect(screen, WHITE, input_rect)
            pygame.draw.rect(screen, DARK_BLUE, input_rect, 2)
            
            # Draw current size
            if size > 0:
                size_text = FONT.render(str(size), True, BLACK)
                screen.blit(size_text, (input_rect.centerx - size_text.get_width() // 2, 
                                      input_rect.centery - size_text.get_height() // 2))
            
            # Draw valid sizes
            valid_sizes = SMALL_FONT.render("Valid sizes: 4, 6, 9", True, GRAY)
            screen.blit(valid_sizes, (SCREEN_WIDTH // 2 - valid_sizes.get_width() // 2, 260))
            
            pygame.display.flip()
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return
                if event.type == pygame.KEYDOWN:
                    if pygame.K_0 <= event.key <= pygame.K_9:
                        digit = int(pygame.key.name(event.key))
                        size = size * 10 + digit
                        # Limit to reasonable sizes
                        if size > 25:
                            size = 25
                    elif event.key == pygame.K_RETURN and size > 0:
                        if size in [4, 6, 9]:
                            GRID_SIZE = size
                            BOARD = handle_input_board(screen, GRID_SIZE)
                            if BOARD is not None:
                                ORIGINAL_BOARD = BOARD.copy()  # Store original board
                            size_inputting = False
                        else:
                            # Flash red to indicate invalid size
                            pygame.draw.rect(screen, RED, input_rect)
                            pygame.display.flip()
                            pygame.time.delay(200)

                    elif event.key == pygame.K_BACKSPACE:
                        size //= 10
                    elif event.key == pygame.K_ESCAPE:
                        return # Go back to main menu
        
        if BOARD is None:
            return # User cancelled input

    elif input_method == '2':
        BOARD, GRID_SIZE = select_predefined_board(screen)
        if BOARD is None:
            return # User cancelled selection
        ORIGINAL_BOARD = BOARD.copy()  # Store original board

    if BOARD is None or GRID_SIZE == 0:
        return

    # Reset variables
    SOLVING = False
    ALGORITHM = None
    BACKTRACKING_ITERATIONS = 0
    GA_GENERATION = 0
    GA_BEST_FITNESS = 0
    GA_BEST_BOARD = None
    SOLVING_SPEED = 1
    SOLUTION_FOUND = False
    SOLUTION_TIME = 0
    SOLUTION_ITERATIONS = 0
    SOLUTION_GENERATIONS = 0
    SOLUTION_ALGORITHM = None

    # Main game loop
    solve_backtrack_button, solve_genetic_button, reset_button = draw_board(screen, BOARD, GRID_SIZE)
    pygame.display.flip()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                if not SOLVING:
                    if solve_backtrack_button.collidepoint(event.pos):
                        SOLVING = True
                        ALGORITHM = 'B'
                        BACKTRACKING_ITERATIONS = 0
                        SOLUTION_FOUND = False
                        START_TIME = time.time()
                        board_copy = BOARD.copy()
                        if solve_backtracking_gui(board_copy, GRID_SIZE, screen):
                            BOARD = board_copy
                        SOLVING = False
                        ALGORITHM = None
                    elif solve_genetic_button.collidepoint(event.pos):
                        SOLVING = True
                        ALGORITHM = 'G'
                        GA_GENERATION = 0
                        GA_BEST_FITNESS = 0
                        GA_BEST_BOARD = None
                        SOLUTION_FOUND = False
                        START_TIME = time.time()
                        ga_solver = SudokuGA_GUI(BOARD.copy(), GRID_SIZE, step_display_interval=GA_STEP_INTERVAL)
                    elif reset_button.collidepoint(event.pos):
                        SOLVING = False
                        ALGORITHM = None
                        SOLUTION_FOUND = False
                        BACKTRACKING_ITERATIONS = 0
                        SOLUTION_TIME = 0
                        SOLUTION_ITERATIONS = 0
                        SOLUTION_GENERATIONS = 0
                        SOLUTION_ALGORITHM = None
                        GA_GENERATION = 0
                        GA_BEST_FITNESS = 0
                        GA_BEST_BOARD = None
                        SOLVING_SPEED = 1

                        # Reload the initial board based on how it was input
                        if input_method == '1':
                            GRID_SIZE = 0
                            BOARD = None
                            main() # Go back to input
                            return
                        elif input_method == '2':
                            BOARD, GRID_SIZE = select_predefined_board(screen)
                            if BOARD is None:
                                return
                            ORIGINAL_BOARD = BOARD.copy()
                else:
                    if reset_button.collidepoint(event.pos):
                        SOLVING = False
                        ALGORITHM = None
                        if input_method == '1':
                            GRID_SIZE = 0
                            BOARD = None
                            main()
                            return
                        elif input_method == '2':
                            BOARD, GRID_SIZE = select_predefined_board(screen)
                            if BOARD is None:
                                return
                            ORIGINAL_BOARD = BOARD.copy()

        if SOLVING and ALGORITHM == 'G':
            result = ga_solver.step()
            if result == False:
                  # Keep solving - display current progress
                GA_GENERATION = ga_solver.generation_count
                GA_BEST_FITNESS = ga_solver.best_fitness
                GA_BEST_BOARD = ga_solver.get_current_best()
                draw_board(screen, GA_BEST_BOARD, GRID_SIZE, algorithm=ALGORITHM, 
                generation=GA_GENERATION, fitness=GA_BEST_FITNESS)
                pygame.display.flip()
                pygame.time.delay(max(1, 100 // SOLVING_SPEED))
            else:
                # Algorithm finished or was interrupted
                SOLVING = False
                ALGORITHM = None
        
            if result == "complete":
                # Solution was found or algorithm completed naturally
                BOARD = ga_solver.get_current_best()
            elif result == "cancel":
                # User pressed ESC - revert to original board
                BOARD = ORIGINAL_BOARD.copy()
                # result == "quit" doesn't need special handling here
        
                draw_board(screen, BOARD, GRID_SIZE)
                pygame.display.flip()
            elif not SOLVING:
                draw_board(screen, BOARD, GRID_SIZE)
                pygame.display.flip()

    pygame.quit()

if __name__ == '__main__':
    main()